<a href="https://colab.research.google.com/github/RichardMbingi/simba-share/blob/master/market_analysis_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [34]:
pip install quandl

In [0]:
#Install dependencies
import quandl
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split

In [36]:
#Get stock data
df = quandl.get("WIKI/AMZN")
#View data
print(df.head())

             Open   High    Low  ...  Adj. Low  Adj. Close  Adj. Volume
Date                             ...                                   
1997-05-16  22.38  23.75  20.50  ...  1.708333    1.729167   14700000.0
1997-05-19  20.50  21.25  19.50  ...  1.625000    1.708333    6106800.0
1997-05-20  20.75  21.00  19.63  ...  1.635833    1.635833    5467200.0
1997-05-21  19.25  19.75  16.50  ...  1.375000    1.427500   18853200.0
1997-05-22  17.25  17.38  15.75  ...  1.312500    1.395833   11776800.0

[5 rows x 12 columns]


In [37]:
#Get Adjusted Close Price
df = df[['Adj. Close']]
#View column
print(df.head())

            Adj. Close
Date                  
1997-05-16    1.729167
1997-05-19    1.708333
1997-05-20    1.635833
1997-05-21    1.427500
1997-05-22    1.395833


In [38]:
# Var for predicting 'n' days out into the future
forecast_out = 30
#Create another column (Dependant Variable) shifted 'n' units up
df['Prediction'] = df[['Adj. Close']].shift(-forecast_out)
#print the new data set
print(df.tail())

            Adj. Close  Prediction
Date                              
2018-03-21     1581.86         NaN
2018-03-22     1544.10         NaN
2018-03-23     1495.56         NaN
2018-03-26     1555.86         NaN
2018-03-27     1497.05         NaN


In [39]:
#Create the independent data set
#Convert the dataframe to a n umpy array
X = np.array(df.drop(['Prediction'],1))
#Remove the last 'n' rows
X = X[:-forecast_out]
print(X)

[[   1.72916667]
 [   1.70833333]
 [   1.63583333]
 ...
 [1350.47      ]
 [1338.99      ]
 [1386.23      ]]


In [40]:
#Create the dependent data set (y) 
#Convert the dataframe to a numpy array (All the values including the NaN's)
y = np.array(df['Prediction'])
#Get all the y values except the last 'n' rows
y = y[:-forecast_out] 
print(y)

[1.54166667e+00 1.51583333e+00 1.58833333e+00 ... 1.49556000e+03
 1.55586000e+03 1.49705000e+03]


In [0]:
#Split the data and 80% training and 20% testing
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [42]:
#Create and train the Support Vector Machine (Regressor)
svr_rbf = SVR(kernel = 'rbf', C = 1e3, gamma = 0.1)
svr_rbf.fit(x_train, y_train)

SVR(C=1000.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma=0.1,
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [43]:
#Testing Model: Score returns the coefficient of the determination R^2 of the prediction.
#The best possible score is 1.0
svm_confidence = svr_rbf.score(x_test, y_test)
print("svm confidence", svm_confidence)

svm confidence 0.9433721455946273


In [44]:
#Create and train the Linear Regression Model
lr = LinearRegression()
#Train the model
lr.fit(x_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [45]:
#Testing Model: Score returns the coefficient of the determination R^2 of the prediction.
#The best possible score is 1.0
lr_confidence = lr.score(x_test, y_test)
print("lr confidence", lr_confidence)

lr confidence 0.9893580546759267


In [46]:
#Set x_forecast equal to the last 30 rows of the original data set from Adj. Close column
x_forecast = np.array(df.drop(['Prediction'],1))[-forecast_out:]
print(x_forecast)

[[1414.51]
 [1451.05]
 [1461.76]
 [1448.69]
 [1468.35]
 [1482.92]
 [1484.76]
 [1500.  ]
 [1521.95]
 [1511.98]
 [1512.45]
 [1493.45]
 [1500.25]
 [1523.61]
 [1537.64]
 [1545.  ]
 [1551.86]
 [1578.89]
 [1598.39]
 [1588.18]
 [1591.  ]
 [1582.32]
 [1571.68]
 [1544.93]
 [1586.51]
 [1581.86]
 [1544.1 ]
 [1495.56]
 [1555.86]
 [1497.05]]


In [47]:
#Print the linear regression model predictions for the next 'n' days
lr_prediction = lr.predict(x_forecast)
print(lr_prediction)

[1499.81663259 1538.65011393 1550.03234122 1536.14198541 1557.03597127
 1572.52047655 1574.47596658 1590.6725253  1614.00024603 1603.40446582
 1603.90396599 1583.7114059  1590.93821688 1615.76443812 1630.67504959
 1638.49700971 1645.78758666 1674.5141603  1695.23810354 1684.38725941
 1687.38426044 1678.15944878 1666.85161513 1638.42261607 1682.61243966
 1677.67057627 1637.54052002 1585.95384284 1650.03865195 1587.53736466]


In [48]:
#Print the support vector regressor model predictions for the next 'n' days
svm_prediction = svr_rbf.predict(x_forecast)
print(svm_prediction)

[1056.32125772  676.45454394  675.46300293  702.84458013  675.46300293
  675.46300293  675.46300293  675.46300293  675.46300293  675.46300293
  675.46300293  675.46300293  675.46300293  675.46300293  675.46300293
  675.46300293  675.46300293  675.46300293  675.46300293  675.46300293
  675.46300293  675.46300293  675.46300293  675.46300293  675.46300293
  675.46300293  675.46300293  675.46300293  675.46300293  675.46300293]
